<a href="https://colab.research.google.com/github/iemio/scratchml/blob/main/notebook_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import Counter

import numpy as np

In [16]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [17]:
data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

In [2]:
!pip install nbimporter

In [3]:
!git clone https://github.com/iemio/scratchml.git

Cloning into 'scratchml'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), done.


In [4]:
%cd scratchml

/content/scratchml


In [5]:
!ls

notebook_01.ipynb  README.md


In [6]:
!jupyter nbconvert --to script --output notebook_01.py notebook_01.ipynb


[NbConvertApp] Converting notebook notebook_01.ipynb to script
[NbConvertApp] Writing 4299 bytes to notebook_01.py.txt


In [38]:
%cd /content/scratchml
from notebook_01 import DecisionTree,accuracy  # Now you can access everything in notebook_01.py which was Decision tree notebook


/content/scratchml


In [29]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]

In [30]:
def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

In [31]:
class RandomForest:
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                min_samples_split=self.min_samples_split,
                max_depth=self.max_depth,
                n_feats=self.n_feats,
            )
            X_samp, y_samp = bootstrap_sample(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        #[111 010 111] --> [101 111 101]
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)

In [37]:
clf = RandomForest(n_trees=3, max_depth=10)

clf.fit(X_train, y_train)


In [36]:
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)

print("Accuracy:", acc)

Accuracy: 0.956140350877193
